In [1]:
import psycopg2

import pandas as pd
import numpy as np

from copy import deepcopy

import ast
import random
import networkx as nx
import time, unicodedata
import itertools

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from joblib import Parallel, delayed

In [2]:
def clean(name, min_len=5, junk_replacement=''):
    try:
        cleaned = unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').lower().decode("ascii")
    except TypeError:
        return junk_replacement
    if len(cleaned) < min_len:
        return junk_replacement
    return cleaned

def get_matches_edit_distance(item, choices, limit, scorer=fuzz.WRatio):
    return process.extract(item, choices, limit=limit, scorer=scorer)
counter = 0
def get_sehir_twitter_matches(twitter_users, sehir_directory, limit=1):
    global fullnames, counter
    twitter_user_by_screen_name = twitter_users.set_index('screen_name')
    start = time.time()
    for screen_name in twitter_users['screen_name']:
        twitter_name = twitter_user_by_screen_name.loc[screen_name]['name']
        match_name = get_matches_edit_distance(twitter_name, fullnames, limit)
        counter += 1
#         if counter %100 == 0:
#             print(counter, "out of ", len(twitter_users))
#             start_ = time.time()
#             print(start_-start, "seconds")
#             start = start_
        yield (screen_name, match_name)
        
def filter_matches_by_threshold(matches_dict, threshold=70):
    filtered_dict = dict()
    for screen_name, matches in matches_dict.items():
        filtered = [(match, score) for match, score in matches if score > threshold]
        
        if filtered:
            filtered_dict[screen_name] = filtered
        
    return filtered_dict

def get_matches_dataframe(twitter_users, sehir_directory, threshold=70, limit=1):
    matches = {screen_name : match_name for screen_name, match_name in 
               get_sehir_twitter_matches(twitter_users, sehir_directory, limit=limit)}
    
    filtered_matches = filter_matches_by_threshold(matches, threshold=threshold)
    screen_names = filtered_matches.keys()
    return pd.DataFrame({'screen_name': list(screen_names),
                         'match_name': [filtered_matches[screen_name] for screen_name in screen_names]})

In [3]:
connection = psycopg2.connect('dbname=link_formation host=localhost user=postgres password=1_sehir_1')

user_connections = pd.read_sql("SELECT * FROM twitter_connection", connection).drop('id', axis=1)

In [4]:
truncate = lambda x: int(str(int(x))[:9])

twitter_users = pd.read_sql("SELECT * FROM twitter_user", connection)
twitter_users = twitter_users.where(twitter_users.match_name.str.len()>6)\
                             .dropna()
twitter_users["truncated_id"] = twitter_users.id.apply(truncate)

#         .where(twitter_users.match_ratio>85)
# .where(~twitter_users.name.str.contains("(?i)sehir"))\
twitter_users.sample(5)

id          name      screen_name   lang  \
18811  4.647111e+08    zeki kafcı        kafcizeki  en-gb   
33615  9.496794e+17         kübra     kbra84254172     en   
23152  9.978261e+17    Selo  dayi  Selahat92466917     en   
59654  1.349113e+09  Tarih Kulübü      TarihSinema     tr   
46982  8.590194e+17          EÇ2💫     _eslemcoban2     tr   

                match_name  match_ratio  followers_count  friends_count  \
18811   Zeki Murat Karakoc         86.0             48.0          158.0   
33615              Library         68.0              0.0           35.0   
23152      Selman Gunaydin         67.0             27.0          466.0   
59654              ! Tarih         90.0           3696.0           29.0   
46982            Erasmus -         45.0            623.0          147.0   

       truncated_id  
18811     464711123  
33615     949679425  
23152     997826081  
59654     134911345  
46982     859019351

In [5]:
is_org = lambda x:"sehir" in clean(x)
twitter_users["is_org"] = twitter_users.screen_name.apply(is_org)
twitter_users.sample(5)

id                  name      screen_name lang  \
48260  9.619998e+17                Yağmur      yagmurakin0   tr   
20731  2.153424e+08         Ramazan YAŞAR           RYASAR   tr   
12361  5.215949e+08      Sinema Bağımlısı  sinemabagimlisi   tr   
50737  7.743138e+17           TuthanKamoN      SalihUygur6   tr   
9828   1.421121e+09  Şehir İşletme Bölümü     sehirisletme   tr   

                                              match_name  match_ratio  \
48260                                      Yagmur Simsek         72.0   
20731                                      Ramazan Hasar         88.0   
12361                            Sinema Televizyon Admin         86.0   
50737                               Safa Batuhan Kamanli         74.0   
9828    Şehir Üniversitesi (İnsan ve Toplum Bilimleri...         86.0   

       followers_count  friends_count  truncated_id  is_org  
48260           2806.0         4523.0     961999779   False  
20731            629.0         2349.0     215342367   False  
12361         726926.0         7535.0     521594852   False  
50737             11.0          229.0     774313785   False  
9828             106.0           15.0     142112088    True

In [6]:
sehir_orgs = twitter_users[twitter_users.is_org==True].set_index("id")
sehir_users = twitter_users[twitter_users.is_org==False]

In [7]:
len(sehir_orgs)

231

In [8]:
sehir_orgs.sample(5)

name      screen_name lang  \
id                                                                             
1.262309e+09                              itiraf şehir      itirafsehir   tr   
4.585214e+08                            Şehir_Münazara   sehir_munazara   tr   
7.530450e+08                         Şehir Gezi Kulübü  sehirgezikulubu   tr   
8.301260e+17  Şehir Üniversitesi Çocuk Gelişimi Kulübü    sehirunicocuk   tr   
3.064906e+09                                GasteŞehir       gastesehir   tr   

                                                     match_name  match_ratio  \
id                                                                             
1.262309e+09   Şehir Üniversitesi (İnsan ve Toplum Bilimleri...         86.0   
4.585214e+08                                     Irem Naz Saral         69.0   
7.530450e+08   Şehir Üniversitesi (İnsan ve Toplum Bilimleri...         86.0   
8.301260e+17                                 Şehir Üniversitesi         90.0   
3.064906e+09                                        Aisec Sehir         70.0   

              followers_count  friends_count  truncated_id  is_org  
id                                                                  
1.262309e+09              1.0           10.0     126230883    True  
4.585214e+08            337.0          111.0     458521438    True  
7.530450e+08            111.0          127.0     753045012    True  
8.301260e+17             48.0           18.0     830126048    True  
3.064906e+09           1198.0           24.0     306490639    True

In [9]:
sehir_users.head(5)

id                 name      screen_name lang  \
1  5.670900e+08          Övünç Meriç       ovuncmeric   tr   
4  7.262076e+08               klasik  klasikyayinlari   tr   
5  4.979428e+08       Küre Yayınları    kureyayinlari   tr   
6  2.674867e+09  Fatıma Tuba Yaylacı  fatimatubapetek   en   
7  1.439589e+09          Mahmut Koca          mkoca66   en   

             match_name  match_ratio  followers_count  friends_count  \
1        Kardelen Meric         77.0            569.0         1170.0   
4         Yavuz Kasikci         75.0           6596.0          142.0   
5        Merve Yakinlar         72.0           9158.0          166.0   
6   Fatima Tuba Yaylaci         94.0            844.0          246.0   
7           Mahmut Koca        100.0           1379.0           44.0   

   truncated_id  is_org  
1     567090020   False  
4     726207614   False  
5     497942798   False  
6     267486658   False  
7     143958858   False

In [10]:
sehir_directory = pd.read_csv('../datasets/contacts.csv', 
                               encoding = "ISO-8859-1", 
                               usecols=['First Name', 'Last Name', 'Primary Email'])
sehir_directory.replace(np.nan, '', regex=True, inplace=True)

In [11]:
fullnames = [' '.join(first_last_name).lower() 
                 for first_last_name in sehir_directory[['First Name', 'Last Name']].values]

In [ ]:
start = time.time()
sehir_matches = Parallel(n_jobs=-1)(delayed(get_matches_dataframe)(
    sehir_users[int(i*(len(sehir_users)/8)):int((i+1)*(len(sehir_users)/8))],
    sehir_directory) for i in range(8))
print("took: ", time.time()-start)

In [ ]:
sehir_matches_df = pd.concat(sehir_matches)
sehir_matches_df.index = range(len(sehir_matches_df))
print("There are {} matches".format(len(sehir_matches_df)))
sehir_matches_df.sample(5)

In [480]:
sehir_matches_df['match_ratio'] = sehir_matches_df.match_name.apply(lambda x: x[0][1])
sehir_matches_df.match_name = sehir_matches_df.match_name.apply(lambda x: x[0][0])
sehir_matches_df.sample(5)

match_name   screen_name  match_ratio
28057    fatma derya mentes   DeryaaSarii           90
10093      nur betã¼l yerli  bacimbilegin           90
20027       emine bayraktar   emine_blt_1           86
19182      etem hakan ergec   hakan_tunaa           86
3016    sami anis abuhamdeh    samiyigit_           86

In [481]:
tu=twitter_users.drop(labels=["match_name","match_ratio"], axis=1)

In [482]:
twitter_users = sehir_matches_df.merge(tu, on="screen_name")
twitter_users = twitter_users.set_index("id")
twitter_users.head(5)

match_name  \
id                                                                
9.970584e+17                                  azize fatma cakir   
9.369498e+17                                        okan mergen   
2.306379e+09   ä°stanbul åehir ãniversitesi ä°åletme enst...   
3.334179e+08                               hanife kubra demirci   
4.742273e+09                                 gizem serpil boylu   

                 screen_name  match_ratio                  name lang  \
id                                                                     
9.970584e+17       fgurbuz35           86          fatma gürbüz   tr   
9.369498e+17      miraokan42           86    miraç okan ekmekci   tr   
2.306379e+09   CHPIstGenclik           86  CHP İstanbul Gençlik   tr   
3.334179e+08   RabiaDeemirci           86         Rabia Demirci   tr   
4.742273e+09  kronik__rehber           86          serpil.sedef   tr   

              followers_count  friends_count  truncated_id  is_org  
id                                                                  
9.970584e+17             68.0          122.0     997058428   False  
9.369498e+17              4.0           19.0     936949783   False  
2.306379e+09           5735.0         3223.0     230637902   False  
3.334179e+08             96.0          314.0     333417875   False  
4.742273e+09             49.0          113.0     474227276   False

In [483]:
sehir_orgs.to_csv("../datasets/orgs.csv",index_label="id")
sehir_orgs.to_csv("../REST/static/orgs.csv",index_label="id")

In [12]:
twitter_users = pd.read_csv("../datasets/twitter_users.csv", index_col="id")
twitter_users.sample(5)

match_name     screen_name  match_ratio  \
id                                                                          
9.728785e+08                     ! sosyoloji  auzefsosyoloji           90   
7.403476e+17  areen munir mohammad altounesi          fm_uni           75   
1.941674e+09              omer faruk ozcetin    farukozcetin           91   
2.442222e+08            meryem ayse gurpinar    AyseOzceyhan           86   
1.483375e+09                     duygu masal     duygubektik           86   

                                  name lang  followers_count  friends_count  \
id                                                                            
9.728785e+08           Auzef Sosyoloji   tr           3335.0            6.0   
7.403476e+17                    Uni FM   tr           1054.0         4307.0   
1.941674e+09        Ömer Faruk Özçetin   tr            129.0          331.0   
2.442222e+08             Ayse Ozceyhan   tr             91.0          142.0   
1.483375e+09  Dr Duygu Bektik (Simsek)   en            464.0          198.0   

              truncated_id  is_org  
id                                  
9.728785e+08     972878497   False  
7.403476e+17     740347593   False  
1.941674e+09     194167361   False  
2.442222e+08     244222222   False  
1.483375e+09     148337542   False

In [32]:
filtered_twu = twitter_users[twitter_users.match_ratio>90]
filtered_twu = filtered_twu.append(twitter_users.loc[291122559])  # me: Ammar Rashed :)

In [33]:
len(twitter_users), len(filtered_twu)

(41768, 2618)

In [35]:
filtered_twu = pd.read_csv("../datasets/filtered_twitter_users.csv", index_col="id")
filtered_twu.sample(5)

match_name      screen_name  match_ratio            name  \
id                                                                           
2.726240e+08   dogukan kotan     dogukankotan           92   Doğukan KOTAN   
7.598048e+17   mustafa celik  _mustafa__celik           96   Mustafa Çelik   
9.572194e+17  nisanur yukruk      nisanurykrk           92  Nisanur Yükrük   
9.281341e+08     emre kaplan            ylmdk          100     Emre Kaplan   
1.576166e+08      esma demir        esmaozdmr           95    Esma Özdemir   

             lang  followers_count  friends_count  truncated_id  is_org  
id                                                                       
2.726240e+08   tr            276.0          139.0     272623984   False  
7.598048e+17   tr              1.0           26.0     759804840   False  
9.572194e+17   tr             55.0          102.0     957219363   False  
9.281341e+08   tr            175.0         1118.0     928134109   False  
1.576166e+08   en            729.0          380.0     157616590   False

In [36]:
filtered_twu.loc[291122559]

match_name            ammar rasid
screen_name        AmmarRashed_MB
match_ratio                    87
name                 Ammar Rashed
lang                           en
followers_count               392
friends_count                 337
truncated_id            291122559
is_org                      False
Name: 291122559.0, dtype: object

In [37]:
filtered_twu[filtered_twu.screen_name.str.contains("kral")]

match_name screen_name  match_ratio           name lang  \
id                                                                         
1.466873e+07    ahmet bulut        kral          100    Ahmet Bulut   en   
2.387189e+09  mehmet yilmaz   kralyoshi           92  Mehmet Yılmaz   tr   

              followers_count  friends_count  truncated_id  is_org  
id                                                                  
1.466873e+07            242.0           62.0      14668733   False  
2.387189e+09              5.0          203.0     238718926   False

# Filter connections

In [38]:
twu_with_orgs = pd.concat([filtered_twu, sehir_orgs])
twu_with_orgs.sample(5)

followers_count  friends_count  is_org lang  \
id                                                          
3.310023e+08             80.0           85.0   False   tr   
3.299537e+09             24.0          208.0    True   tr   
2.537778e+09            293.0          412.0   False   tr   
3.172710e+08             62.0          324.0   False   tr   
8.258187e+08             35.0          156.0   False   tr   

                                                     match_name  match_ratio  \
id                                                                             
3.310023e+08                                        elif kartal         95.0   
3.299537e+09   Şehir Üniversitesi (İnsan ve Toplum Bilimleri...         86.0   
2.537778e+09                                        yusuf sahan         91.0   
3.172710e+08                                       halime demir        100.0   
8.258187e+08                                        mehmet kara         91.0   

                       name screen_name  truncated_id  
id                                                     
3.310023e+08   elif karaçöl   cakkannim     331002312  
3.299537e+09  SCORP - Şehir  Scorpsehir     329953707  
2.537778e+09    Yusuf Şahan    ysfsahan     253777838  
3.172710e+08   Halime Demir    Rosican1     317270977  
8.258187e+08    Mehmet ARDA   memisarda     825818708

In [39]:
assert len(twu_with_orgs) == len(filtered_twu) + len(sehir_orgs)
len(twu_with_orgs)

2849

In [41]:
user_connections.formation = user_connections.formation.apply(lambda x:{"2018.05.24":True})
user_connections.sample(5)

from_user_id  to_user_id             formation
32188           339133152   458521438  {'2018.05.24': True}
80823           158783378   995806488  {'2018.05.24': True}
92443           281127185   163020501  {'2018.05.24': True}
71572           417842607   618131962  {'2018.05.24': True}
73092  799523617050656769    32660008  {'2018.05.24': True}

In [42]:
ids = set(twu_with_orgs.index)
def in_sehir(row, from_col="from_user_id", to_col="to_user_id"):
    return row[from_col] in ids and row[to_col] in ids

In [43]:
user_connections["in_sehir"] = user_connections.apply(lambda row: in_sehir(row), axis=1)
sehir_connections = user_connections[user_connections.in_sehir].drop("in_sehir", axis=1)
sehir_connections.sample(5)

from_user_id          to_user_id             formation
53782     428560048           377901136  {'2018.05.24': True}
34228     287084927           609301446  {'2018.05.24': True}
22209     146166034           443551180  {'2018.05.24': True}
39386    2360133031  839025106894602240  {'2018.05.24': True}
6544     3408479039           106086098  {'2018.05.24': True}

In [44]:
len(user_connections),len(sehir_connections)

(93296, 9020)

## Adding older connections

In [45]:
from datetime import datetime
def get_dates(cons):
    all_dates = set()
    str2date = lambda strdate: datetime.strptime(strdate, '%Y.%m.%d')  # 2018.05.08

    for dates in cons.formation.apply(lambda x: list(x)):
        for date in dates:
            all_dates.add(str2date(date))
    return [d.strftime('%Y.%m.%d') for d in sorted(all_dates)]

def present_in_date(changes_dates, queried_date):
    """
    changes_dates = {d1:True, d2:False, d3:True} connection added or removed
    """
    if changes_dates:
        str2date = lambda strdate: datetime.strptime(strdate, '%Y.%m.%d')  # 2018.05.08
        changes = sorted(changes_dates,key=lambda d: str2date(d))
        queried_date = datetime.strptime(queried_date, '%Y.%m.%d')
        present = False
        for d in changes:
            if queried_date < str2date(d):
                break
            present = changes_dates[d]
        return present
    else:
        return False

In [46]:
con2 = psycopg2.connect('dbname=old host=localhost user=postgres password=1_sehir_1')

old_cons = pd.read_sql("SELECT * FROM twitter_connection", con2).drop('id', axis=1)

In [47]:
old_cons.sample(5)

from_user_id          to_user_id             formation
13564  700797513188712450          3892757176  {'2018.05.08': True}
24581          2960439239          1536995378  {'2018.05.08': True}
29858           887437856  793166250486730753  {'2018.05.08': True}
30683            73543596          4091594428  {'2018.05.08': True}
13578          3007709429          3892757176  {'2018.05.08': True}

In [48]:
old_cons["in_sehir"] = old_cons.apply(lambda row: in_sehir(row), axis=1)
old_sehir = old_cons[old_cons.in_sehir].drop("in_sehir", axis=1)
old_sehir.sample(5)

from_user_id  to_user_id             formation
20536    1110823566   293976465  {'2018.05.08': True}
21174    2301123770  1110823566  {'2018.05.08': True}
32853    4217413882   737056442  {'2018.05.08': True}
6924      107674642   106086098  {'2018.05.08': True}
36118     450639507   149522762  {'2018.05.08': True}

In [49]:
len(old_cons),len(old_sehir)

(39848, 6200)

In [50]:
concat_cons = pd.concat([sehir_connections, old_sehir])
assert len(concat_cons) == len(old_sehir)+len(sehir_connections)
len(concat_cons)

15220

In [51]:
def optimize_dates(dates_):
    dates = {list(d.keys())[0]:d[list(d.keys())[0]] for d in dates_}
    str2date = lambda strdate: datetime.strptime(strdate, '%Y.%m.%d')  # 2018.05.08
    sorted_dates = sorted(dates, key=lambda d:str2date(d))
    optimized_dates = {sorted_dates[0]: True}
    for d in range(1, len(sorted_dates)):
        if dates[sorted_dates[d-1]] != dates[sorted_dates[d]]:
            optimized_dates[sorted_dates[d]] = dates[sorted_dates[d]]
    return str(optimized_dates)

In [52]:
grouped_cons = concat_cons.groupby(["from_user_id","to_user_id"])["formation"]\
                    .apply(lambda x:optimize_dates(x)).reset_index()
len(grouped_cons)

9031

In [53]:
len(old_sehir), len(sehir_connections)

(6200, 9020)

In [54]:
str2dict = lambda d : ast.literal_eval(d)
grouped_cons.formation = grouped_cons.formation.apply(lambda d: str2dict(d))
grouped_cons.sample(5)

from_user_id  to_user_id             formation
7881  700707449079201792  1666891914  {'2018.05.08': True}
7492          3892757176   194475532  {'2018.05.08': True}
793            162427198  4117422969  {'2018.05.08': True}
3037           584303296   106086098  {'2018.05.08': True}
2514           450639507   609301446  {'2018.05.08': True}

In [55]:
twu_with_orgs.to_csv("../datasets/filtered_twitter_users.csv", index_label="id")
twu_with_orgs.to_csv("../REST/static/filtered_twitter_users.csv", index_label="id")

grouped_cons.to_csv("../datasets/filtered_twitter_connections.csv", index_label="id")
grouped_cons.to_csv("../REST/static/filtered_twitter_connections.csv", index_label="id")

## Construct the network

In [56]:
dates = get_dates(grouped_cons)
dates

['2018.05.08', '2018.05.24']

In [57]:
grouped_cons["first_date"] = grouped_cons.formation.apply(
    lambda ds: present_in_date(ds, dates[0]))
grouped_cons.sample(5, random_state=42)

from_user_id  to_user_id             formation  first_date
2373     450639507   151606317  {'2018.05.08': True}        True
7802    4576344862   564388225  {'2018.05.08': True}        True
5123    1727958896   174415744  {'2018.05.08': True}        True
6108    2590180702  1222491402  {'2018.05.08': True}        True
6011    2529427087   609301446  {'2018.05.08': True}        True

In [58]:
G = nx.DiGraph()
# for _, row in user_connections.iterrows():
for _, row in grouped_cons[grouped_cons.first_date==True].iterrows():    
    from_ = truncate(row["from_user_id"])
    to = truncate(row["to_user_id"])
    if from_ in twu_with_orgs.truncated_id and to in twu_with_orgs.truncated_id:
        G.add_edge(from_, to)

In [59]:
augs = ["name", "screen_name","match_name", "followers_count","friends_count", "lang"]
for node in G.nodes():
    user = twu_with_orgs.loc[node]
    for aug in augs:
        if aug=="lang":
            m = user[aug]
        elif type(user[aug])==str:
            m = clean(user[aug])
        else:
            m = user[aug]
        G.nodes[node][aug] = m

In [60]:
len(G.nodes())

722

In [61]:
len(G.edges())

1280

In [62]:
for ix,deg in G.degree(G.nodes()):
    G.node[ix]['degree'] = deg
    G.node[ix]['parity'] = (1-deg%2)
    
for ix,in_deg in G.in_degree(G.nodes()):
    G.node[ix]['in_degree'] = in_deg
    
for ix,out_deg in G.out_degree(G.nodes()):
    G.node[ix]['out_degree'] = out_deg

In [63]:
evc = nx.eigenvector_centrality(G)
closeness = nx.closeness_centrality(G)
betweenness = nx.betweenness_centrality(G)
pagerank = nx.pagerank(G)
nxg = G.to_undirected()
clustering = nx.clustering(nxg)

In [64]:
metrics = {"eigenvector_centrality":evc,
           "closeness_centrality":closeness,
          "betweenness":betweenness,
          "pagerank":pagerank,
          "clustering_coefficient":clustering}

In [65]:
for metric_name, metric in metrics.items():
    for ix,v in metric.items():
        G.nodes[ix][metric_name] = v

In [66]:
list(G.nodes(data=True))[0]

(396662786,
 {'betweenness': 0.0,
  'closeness_centrality': 0.0,
  'clustering_coefficient': 0,
  'degree': 1,
  'eigenvector_centrality': 6.741618620868637e-27,
  'followers_count': 33.0,
  'friends_count': 284.0,
  'in_degree': 0,
  'lang': 'tr',
  'match_name': ' sehir mba',
  'name': 'nemasehir',
  'out_degree': 1,
  'pagerank': 0.0002640177962431144,
  'parity': 0,
  'screen_name': 'nemasehir'})

In [67]:
import json
from networkx.readwrite import json_graph
data = nx.node_link_data(G)
with open('../REST/static/networks/twitter_users_graph2.json', 'w') as f:
    json.dump(data, f, indent=4)

## Calculating Homophily

In [68]:
def homophily(nw, metric="lang"):
    langs_probs = dict()
    for n in nw.nodes():
        user = nw.nodes[n]
        langs_probs.setdefault(user[metric], 0)
        langs_probs[user[metric]] += 1
    heterogeneity_fraction_norm = 1 - sum(
        [(float(i)/len(nw.nodes()))**2 for i in langs_probs.values()])
    cross_edges = sum(
        [int(nw.nodes[f][metric] != nw.nodes[t][metric] ) for f,t in nw.edges()])
    cross_metric_ratio = cross_edges/float(len(nw.edges()))
    print("cross-metric edges ratio: ", cross_metric_ratio)
    print("Heterogeneity Fraction Norm", heterogeneity_fraction_norm)
    return cross_metric_ratio < heterogeneity_fraction_norm

In [69]:
homophily(G)

cross-metric edges ratio:  0.4578125
Heterogeneity Fraction Norm 0.39950199891038285


False

# Transitivity

In [70]:
nx.transitivity(G)

0.024470693113438823